In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from joblib import dump
import pandas as pd
from joblib import dump
import numpy as np

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

C:\Users\86188\AppData\Local\Temp\ipykernel_40004\3611612051.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
fraud = pd.read_csv("https://cs307.org/lab-06/data/fraud.csv")
fraud_train, fraud_test = train_test_split(
    fraud,
    test_size=0.20,
    random_state=42,
    stratify=fraud["Fraud"],
)

# create X and y for train
X_train = fraud_train.drop("Fraud", axis=1)
y_train = fraud_train["Fraud"]

# create X and y for test
X_test = fraud_test.drop("Fraud", axis=1)
y_test = fraud_test["Fraud"]

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, make_scorer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic_regression', LogisticRegression(max_iter=10000))
])

param_grid = {
    'logistic_regression__C': np.logspace(-3, 1, 100)
}
precision_scorer = make_scorer(precision_score)
recall_scorer = make_scorer(recall_score)

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=100,
    scoring='recall',
    verbose=1,
)

grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)

y_pred = grid_search.predict(X_test)

Fitting 100 folds for each of 100 candidates, totalling 10000 fits
Best parameters:  {'logistic_regression__C': 0.15199110829529347}


In [44]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [55]:
from sklearn.metrics import precision_score, recall_score
y_proba = grid_search.predict_proba(X_test)[:, 1] 

threshold = 0.35 

y_pred_custom_threshold = (y_proba >= threshold).astype(int)


precision = precision_score(y_test, y_pred_custom_threshold)
recall = recall_score(y_test, y_pred_custom_threshold)

print("Precision with custom threshold:", precision)
print("Recall with custom threshold:", recall)

Precision with custom threshold: 0.8923076923076924
Recall with custom threshold: 0.7341772151898734


In [42]:
dump(grid_search, "credit-fraud.joblib")

['credit-fraud.joblib']